# Geo analytics AAA time series - Intentionally Blank

The census tract borders and the points of interest data were sourced from the following sites:
- census tract borders: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2016&layergroup=Census+Tracts
- points of interest data: https://overpass-turbo.eu/s/1yIY
There is no need to manually download the files. Both files should be already in the data folder provided be the previously mentioned sciebo link.
The notebook is split into two parts. This notebook visualizes static non time dependend census tract and hexagon bin data while the second notebook visualizes different temporal bin sizes.

**Dependencies needed for this notebook:**
- Pandas
- Numpy
- H3-py (*conda install -c conda-forge h3-py*)
- Geopandas (*conda install -c conda-forge geopandas*)
  - Shapely (*conda install -c conda-forge shapely*)
- Pyarrow (*conda install pyarrow*)
  - Needed for reading the taxi dataset from the parquet file
- Plotly (*conda install -c plotly plotly=5.16.0*)

In [ ]:
import json

import pandas as pd
import numpy as np

import h3
import geopandas as gpd
import shapely
from shapely.geometry import shape
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

## Imports and preparation

First we import the needed files for the analysis.

In [ ]:
# Read taxi data and census tract borders
taxi_df = pd.read_parquet('data/prepared/taxi_data_prepared.gzip')
census_tract_borders = gpd.read_file('data/chicago_census_tract_borders.zip')

Afterwards we do some small preparation steps needed for later use.

In [ ]:
# Change pickup and dropoff location to geopandas geometry objects
taxi_df['pickup_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df['pickup_centroid_location'])
taxi_df['dropoff_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df['dropoff_centroid_location'])

# Drop columns that are not needed for the analysis
census_tract_borders = census_tract_borders.drop(census_tract_borders.columns.difference(['GEOID', 'geometry']), axis=1)

# Get all unique census tract ids from the taxi data and filter the census tract borders to only include those
unique_census_tract_id = np.append(taxi_df['pickup_census_tract'].unique(), taxi_df['dropoff_census_tract'].unique()).astype('str')
census_tract_borders= census_tract_borders[census_tract_borders['GEOID'].isin(unique_census_tract_id)].reset_index(drop=True)

## Helper Functions

The following helper function is similar to the previous helper functions for the census tract borders and the hexagon bins from the first part of the geo analysis. The main difference is that in addition to the location the data is also grouped by the specified time interval.

In [ ]:
def plotByFeatureTime(dataframe, location='pickup', feature='all', aggregation='sum', time_interval='hour', start_time = True):
    """ Filter a feature or all features of a dataframe. Furthermore aggregate the data by location and feature and group by location and time_interval.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    time_interval : (str)
        The time interval to group by. Can be either 'hour', 'month' or 'weekday'. Default is 'hour'.
    missingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.

    Returns
    ----------

    dataframe_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    dataframe_grouped = dataframe.copy()
    dataframe_grouped.dropna(inplace = True)
    if feature == 'all':
        features = dataframe_grouped.columns.difference(['pickup_census_tract', 'dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location']).tolist()
    else:
        features = [feature]
    
    if location == 'pickup':
        features.append('pickup_census_tract')
        features.append('pickup_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=['dropoff_census_tract', 'dropoff_centroid_location'])
    elif location == 'dropoff':
        features.append('dropoff_census_tract')
        features.append('dropoff_centroid_location')
        dataframe_grouped = dataframe_grouped.drop(columns=['pickup_census_tract', 'pickup_centroid_location'])
    else:
        raise ValueError("Location must be either 'pickup' or 'dropoff'.")

    if start_time == True:
        dataframe_grouped.rename(columns={'trip_start_timestamp': 'timestamp'}, inplace=True)
        dataframe_grouped.drop(columns=['trip_end_timestamp'], inplace=True)
    else:
        dataframe_grouped.rename(columns={'trip_end_timestamp': 'timestamp'}, inplace=True)
        dataframe_grouped.drop(columns=['trip_start_timestamp'], inplace=True)
    features[0]='timestamp'
    if time_interval == 'hour':
        dataframe_grouped["hour"] = dataframe_grouped['timestamp'].dt.hour
        dataframe_grouped.rename(columns={'timestamp': 'trip_count'}, inplace=True)
        dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location', 'hour']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    elif time_interval == 'month':
        dataframe_grouped["month"] = dataframe_grouped['timestamp'].dt.month
        dataframe_grouped.rename(columns={'timestamp': 'trip_count'}, inplace=True)
        dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location', 'month']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    elif time_interval == 'weekday':
        dataframe_grouped["weekday"] = dataframe_grouped['timestamp'].dt.weekday
        dataframe_grouped.rename(columns={'timestamp': 'trip_count'}, inplace=True)
        dataframe_grouped = dataframe_grouped.groupby([location + '_census_tract', location + '_centroid_location', 'weekday']).agg(lambda column: column.agg('count') if column.name == 'trip_count' else column.agg(aggregation)).reset_index()
    dataframe_grouped["trip_count"] =dataframe_grouped["trip_count"].astype("int32")
    dataframe_grouped = dataframe_grouped.rename(columns={location + '_census_tract': 'GEOID'})
    dataframe_grouped['GEOID'] = dataframe_grouped['GEOID'].astype('str')
    dataframe_grouped = dataframe_grouped.merge(census_tract_borders, on='GEOID', how='left')
    dataframe_grouped = gpd.GeoDataFrame(dataframe_grouped)
    dataframe_grouped.sort_values(by=[time_interval], inplace=True)
    return dataframe_grouped

In [ ]:
def plotH3_HexagonMapInteractive(dataframe, location='pickup', feature='all', aggregation='sum', hexRes = 10, time_interval='hour'):
    """ Plot a feature of a dataframe on a map.

    Parameters
    ----------

    dataframe :  (pandas.DataFrame) 
        The dataframe to plot.
    location : (str) 
        The location column of the dataframe. Can be either 'pickup' or 'dropoff'. Default is 'pickup'.
    feature : (str) 
        The feature to aggregate. If 'all', all features are aggregated. Default is 'all'
    aggregation : (str)  
        The aggregation function to use. Can be either 'mean', 'median', 'sum', 'count', 'min', 'max'. Default is 'sum'.
    containMissingCensusTract : (bool)
        If True, census tracts with no data are included in the plot. Default is False.
    hexRes : (int)
        H3 hexagon resolution size. Default is 10.
    Returns
    ----------

    taxi_df_geo_grouped : (geopandas.GeoDataFrame) 
        The geodataframe grouped by the location column and the feature column. Contains always a geometry column and trip_count column.
    """
    taxi_df_geo = plotByFeatureTime(dataframe, location= location,feature = feature, aggregation = aggregation, time_interval=time_interval)
    # geometry to h3 index
    taxi_df_geo['h3_index'] = taxi_df_geo.apply(lambda row: h3.geo_to_h3(row[location + '_centroid_location'].y, row[location + '_centroid_location'].x, hexRes), axis=1)

    geojson = []
    geometries = []
    indexes = []

    for geometry in taxi_df_geo['geometry']:
        geojson.append(shapely.to_geojson(geometry))

    for geometry in geojson:
        obj = json.loads(geometry)
        h3_indexes = h3.polyfill(obj, hexRes ,True)
        for index in h3_indexes:
            geometries.append(shape({"type": "Polygon",
                    "coordinates": [h3.h3_to_geo_boundary(index, geo_json=True)],
                    "properties": ""
                    }))
            indexes.append(index)   
    if location == "pickup":
        taxi_df_geo.drop(columns= ['geometry', 'pickup_centroid_location', 'GEOID'], inplace = True)
    else:
        taxi_df_geo.drop(columns= ['geometry', 'dropoff_centroid_location', 'GEOID'], inplace = True)
    df_h3_polyfilled = pd.DataFrame({'h3_index': indexes})
    taxi_df_geo_grouped = taxi_df_geo.groupby(['h3_index', time_interval]).agg(aggregation).reset_index()
    taxi_df_geo_grouped = taxi_df_geo_grouped.merge(df_h3_polyfilled, on='h3_index', how='outer')
    taxi_df_geo_grouped['geometry'] = taxi_df_geo_grouped.apply(lambda row: shape({"type": "Polygon",
                                           "coordinates": [h3.h3_to_geo_boundary(row["h3_index"], geo_json=True)],
                                           "properties": ""
                                           }), axis=1)
    taxi_df_geo_grouped = gpd.GeoDataFrame(taxi_df_geo_grouped, crs='EPSG:4326', geometry='geometry')
    taxi_df_geo_grouped.sort_values(by=[time_interval], inplace=True)
    return taxi_df_geo_grouped

## Analysis of the hourly time intervals

First we plot the trip count for the dropoff location and pickup location next to each other. To compare time dependent patterns between both plots and analysing correlations between the two. The animation feature allowed us to observe how taxi demand evolves throughout the day or over the course of the year.

In [ ]:
interactive_Plot_Pickup_hourly = plotByFeatureTime(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='hour', start_time = True)

In [ ]:
interactive_Plot_Dropoff_hourly = plotByFeatureTime(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='hour', start_time = True)

In [ ]:
viridis = px.colors.sequential.Viridis
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hourly,
                        geojson=interactive_Plot_Pickup_hourly[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactive_Plot_Pickup_hourly.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each hour of day - Census tract borders', title_x=0.5)
fig.show()
fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hourly,
                        geojson=interactive_Plot_Dropoff_hourly[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactive_Plot_Dropoff_hourly.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each hour of day - Census tract borders', title_x=0.5)
fig.show()

Looking at the map with the census tracts, we can observe that at all times the trip count is highest near the center but it is difficult to compare the pickup and dropoff trip count and the difference between different times. Because the shapes are not uniform and of the same size it is difficult to observe more without making misinterpretations. We will look at the data grouped by hexagon bins of a lower and higher resolution to gain more insights.

In [ ]:
interactive_Plot_Pickup_hexRes8_hour = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='hour', hexRes=8)

In [ ]:
interactive_Plot_Dropoff_hexRes8_hour = plotH3_HexagonMapInteractive(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='hour', hexRes=8)

In [ ]:
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hexRes8_hour,
                        geojson=interactive_Plot_Pickup_hexRes8_hour[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Pickup_hexRes8_hour.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each hour of day - Hexagon resolution 8 borders', title_x=0.5)
fig.show()

fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hexRes8_hour,
                        geojson=interactive_Plot_Dropoff_hexRes8_hour[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Dropoff_hexRes8_hour.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each hour of day - Hexagon resolution 8 borders', title_x=0.5)
fig.show()

The same observations can be made for the hexagon bins if we take a resolution of 8. The difference of smaller areas between different time periods is observable but quantifying the overall pattern of a larger area is more difficult with finer resolution.

Below we will look at a plot with a lower resolution to observe the data on a broader view.

In [ ]:
interactive_Plot_Pickup_hexRes6_hour = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='hour', hexRes=6)

In [ ]:
interactive_Plot_Dropoff_hexRes6_hour = plotH3_HexagonMapInteractive(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='hour', hexRes=6)

In [ ]:
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hexRes6_hour,
                        geojson=interactive_Plot_Pickup_hexRes6_hour[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Pickup_hexRes6_hour.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 900000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each hour of day - Hexagon resolution 6 borders', title_x=0.5)
fig.show()

fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hexRes6_hour,
                        geojson=interactive_Plot_Dropoff_hexRes6_hour[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Dropoff_hexRes6_hour.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='hour',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 900000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each hour of day - Hexagon resolution 6 borders', title_x=0.5)
fig.show()

As we can observe the trip count in the center of chicago increases between 8AM and 9AM for the dropoff locations and nearly doubles in comparison to 7AM while for the same time frame the trip count for the pickup location is lower compared to the dropoff trip count and other time periods but is increasing over the hours. Looking at the trip count from the pickup location map, we can see a similar pattern between 16PM and 18PM where the trip count peaks in comparison to the other times while for the same time frame the trip count for the dropoff location is relatively low. The mentioned timeframes correspond with the rush hours. It makes sense that before 8 AM the trip count is low because most people are still sleeping or still at home at those times. At around 18 PM most people have finished working and are going home. Other factors for the high demand could be that most people arrive at the airport or the Navy Pier late in the evening, people meeting up in the morning till afternoon or the generally higher traffic in the city at the mid hours.

## Analysis of the monthly time intervals

Now we will look at the trip count depending on the month of year to find seasonal patterns.

In [ ]:
interactive_Plot_Pickup_monthly = plotByFeatureTime(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='month', start_time = True)

In [ ]:
interactive_Plot_Dropoff_monthly = plotByFeatureTime(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='month', start_time = True)

In [ ]:
viridis = px.colors.sequential.Viridis
fig = px.choropleth_mapbox(interactive_Plot_Pickup_monthly,
                        geojson=interactive_Plot_Pickup_monthly[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactive_Plot_Pickup_monthly.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='month',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each month of year - Census tract borders', title_x=0.5)
fig.show()
fig = px.choropleth_mapbox(interactive_Plot_Dropoff_monthly,
                        geojson=interactive_Plot_Dropoff_monthly[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactive_Plot_Dropoff_monthly.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='month',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each month of year - Census tract borders', title_x=0.5)
fig.show()

In [ ]:
interactive_Plot_Pickup_hexRes8_month = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='month', hexRes=8)

In [ ]:
interactive_Plot_Dropoff_hexRes8_month = plotH3_HexagonMapInteractive(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='month', hexRes=8)

In [ ]:
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hexRes8_month,
                        geojson=interactive_Plot_Pickup_hexRes8_month[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Pickup_hexRes8_month.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='month',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each month of year - Hexagon resolution 8 borders', title_x=0.5)
fig.show()
fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hexRes8_month,
                        geojson=interactive_Plot_Dropoff_hexRes8_month[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Dropoff_hexRes8_month.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='month',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 300000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each month of year - Hexagon resolution 8 borders', title_x=0.5)
fig.show()

In [ ]:
interactive_Plot_Pickup_hexRes6_month = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='month', hexRes=6)

In [ ]:
interactive_Plot_Dropoff_hexRes6_month = plotH3_HexagonMapInteractive(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='month', hexRes=6)

In [ ]:
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hexRes6_month,
                        geojson=interactive_Plot_Pickup_hexRes6_month[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Pickup_hexRes6_month.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='month',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 1200000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each month of year - Hexagon resolution 6 borders', title_x=0.5)
fig.show()
fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hexRes6_month,
                        geojson=interactive_Plot_Dropoff_hexRes6_month[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Dropoff_hexRes6_month.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='month',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 1200000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each month of year - Hexagon resolution 6 borders', title_x=0.5)
fig.show()

Comparing both the dropoff and pickup plots we can observe no significant difference in the distribution of the trip count between both. Moreover the trip count does not significantly change over the month except january, where it drops about about a fifth of the average trip count, and october where it peaks. The trip count for the pickup location is a bit higher for all month in comparison to the trip count for the dropoff location in the same month.

## Analysis of the week of day time intervals

For the last maps we will look at the weekdays to find differences between weekdays and the weekend.

In [ ]:
interactive_Plot_Pickup_weekday = plotByFeatureTime(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='weekday', start_time = True)

In [ ]:
interactive_Plot_Dropoff_weekday = plotByFeatureTime(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='weekday', start_time = True)

In [ ]:
viridis = px.colors.sequential.Viridis
fig = px.choropleth_mapbox(interactive_Plot_Pickup_weekday,
                        geojson=interactive_Plot_Pickup_weekday[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactive_Plot_Pickup_weekday.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='weekday',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 500000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by pickup location for each weekday - Census tract borders', title_x=0.5)
fig.show()
fig = px.choropleth_mapbox(interactive_Plot_Pickup_weekday,
                        geojson=interactive_Plot_Pickup_weekday[['GEOID', 'geometry']].drop_duplicates(subset=['GEOID']), 
                        locations=interactive_Plot_Pickup_weekday.GEOID,
                        featureidkey="properties.GEOID",
                        color='trip_count',
                        animation_frame='weekday',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 500000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=1000,
                        height=1000)
fig.update_layout(title_text='Trip count by dropoff location for each weekday - Census tract borders', title_x=0.5)
fig.show()

In [ ]:
interactive_Plot_Pickup_hexRes8_weekday = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='weekday', hexRes=8)

In [ ]:
interactive_Plot_Dropoff_hexRes8_weekday = plotH3_HexagonMapInteractive(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='weekday', hexRes=8)

In [ ]:
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hexRes8_weekday,
                        geojson=interactive_Plot_Pickup_hexRes8_weekday[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Pickup_hexRes8_weekday.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='weekday',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 500000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.update_layout(title_text='Trip count by pickup location for each weekday - Hexagon resolution 8 borders', title_x=0.5)
fig.show()

fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hexRes8_weekday,
                        geojson=interactive_Plot_Dropoff_hexRes8_weekday[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Dropoff_hexRes8_weekday.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='weekday',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 500000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.update_layout(title_text='Trip count by dropoff location for each weekday - Hexagon resolution 8 borders', title_x=0.5)
fig.show()

In [ ]:
interactive_Plot_Pickup_hexRes6_weekday = plotH3_HexagonMapInteractive(taxi_df, location='pickup', feature='all', aggregation='sum', time_interval='weekday', hexRes=6)

In [ ]:
interactive_Plot_Dropoff_hexRes6_weekday = plotH3_HexagonMapInteractive(taxi_df, location='dropoff', feature='all', aggregation='sum', time_interval='weekday', hexRes=6)

In [ ]:
fig = px.choropleth_mapbox(interactive_Plot_Pickup_hexRes6_weekday,
                        geojson=interactive_Plot_Pickup_hexRes6_weekday[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Pickup_hexRes6_weekday.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='weekday',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 2200000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.update_layout(title_text='Trip count by pickup location for each weekday - Hexagon resolution 6 borders', title_x=0.5)
fig.show()

fig = px.choropleth_mapbox(interactive_Plot_Dropoff_hexRes6_weekday,
                        geojson=interactive_Plot_Dropoff_hexRes6_weekday[['h3_index', 'geometry']].drop_duplicates(subset=['h3_index']), 
                        locations=interactive_Plot_Dropoff_hexRes6_weekday.h3_index,
                        featureidkey="properties.h3_index",
                        color='trip_count',
                        animation_frame='weekday',
                        mapbox_style='open-street-map', 
                        zoom=9.5,
                        color_continuous_scale=[
                            [0, viridis[0]],
                            [1./1000000, viridis[2]],
                            [1./10000, viridis[4]],
                            [1./100, viridis[7]],
                            [1., viridis[9]],
                        ],
                        opacity=0.5, 
                        range_color=[0, 2200000],
                        center={'lat': 41.84, 'lon': -87.723177},
                        labels={'trip_count':'trip_count'},
                        width=800,
                        height=800)
fig.update_layout(title_text='Trip count by dropoff location for each weekday - Hexagon resolution 6 borders', title_x=0.5)
fig.show()

Like in the maps for the month of year and hour of day we can observe the same patterns for example the trip counts for the pickup locations being higher in the center compared to the dropoff locations and the center being more demanded in general regardless of time. Furthermore from the weekdays we can see that friday having the most demand and saturday and especially sunday having the lowest demand. From monday on the demand is increasing over the week till friday at its peak. The low demand on Sunday can be explained by the fact that most people have their offday. In addition, most of the stores are closed and most people likely arrive in Chicago at the beginning of the weekend.